In [20]:
import time
!pip install tweet-preprocessor

In [1]:
import pandas as pd
import preprocessor as p
import nltk
nltk.download('punkt')
from nltk.tokenize import RegexpTokenizer
from nltk import word_tokenize
from nltk.corpus import stopwords
import googletrans
from googletrans import Translator

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\tosin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
en_stopwords = stopwords.words('english')

def remove_stopwords(text):
    result = []
    for token in text:
        if token not in en_stopwords:
            result.append(token)

    return result

LookupError: 
**********************************************************************
  Resource [93mstopwords[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('stopwords')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/stopwords[0m

  Searched in:
    - 'C:\\Users\\tosin/nltk_data'
    - 'C:\\Users\\tosin\\miniconda3\\nltk_data'
    - 'C:\\Users\\tosin\\miniconda3\\share\\nltk_data'
    - 'C:\\Users\\tosin\\miniconda3\\lib\\nltk_data'
    - 'C:\\Users\\tosin\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************


In [3]:
def remove_punct(text):
    tokenizer = RegexpTokenizer(r"\w+")
    lst=tokenizer.tokenize(' '.join(text))
    return lst

In [4]:
data = pd.read_table("assignment-comp3222-comp6246-mediaeval2015-dataset/mediaeval-2015-trainingset.txt", sep="\t",header=0).dropna()
print(data.head())

              tweetId                                          tweetText  \
0  263046056240115712  ¿Se acuerdan de la película: “El día después d...   
1  262995061304852481  @milenagimon: Miren a Sandy en NY!  Tremenda i...   
2  262979898002534400  Buena la foto del Huracán Sandy, me recuerda a...   
3  262996108400271360     Scary shit #hurricane #NY http://t.co/e4JLBUfH   
4  263018881839411200  My fave place in the world #nyc #hurricane #sa...   

      userId      imageId(s)        username                       timestamp  \
0   21226711  sandyA_fake_46         iAnnieM  Mon Oct 29 22:34:01 +0000 2012   
1  192378571  sandyA_fake_09  CarlosVerareal  Mon Oct 29 19:11:23 +0000 2012   
2  132303095  sandyA_fake_09     LucasPalape  Mon Oct 29 18:11:08 +0000 2012   
3  241995902  sandyA_fake_29     Haaaaarryyy  Mon Oct 29 19:15:33 +0000 2012   
4  250315890  sandyA_fake_15  princess__natt  Mon Oct 29 20:46:02 +0000 2012   

  label  
0  fake  
1  fake  
2  fake  
3  fake  
4  fake  


In [11]:
import time
translator = Translator()
translator.raise_Exception = True
p.set_options(p.OPT.URL,p.OPT.EMOJI,p.OPT.RESERVED)
data.tweetText = data.tweetText.apply(lambda x :translator.translate(p.clean(x).lower().replace('\\n',''),dest='en'))
#for sentence in data.tweetText:
   # translator.translate(sentence, dest='en')
data['tokens'] = data.tweetText.apply(lambda x : word_tokenize(x))
list(data.tweetText)

KeyboardInterrupt: 

In [8]:
print(googletrans.__version__)

3.1.0-alpha
